In [1]:
import spacy
import csv
import re
import pandas as pd

nlp = spacy.load("de_core_news_sm")

In [4]:
text = "Die, die die, die die Dietriche erfunden haben, verurteilen, irren sich."
doc = nlp(text)
for token in doc:
    print([token.text, token.lemma_, token.pos_, token.tag_])

['Die', 'der', 'PRON', 'PDS']
[',', ',', 'PUNCT', '$,']
['die', 'der', 'PRON', 'PRELS']
['die', 'der', 'PRON', 'PDS']
[',', ',', 'PUNCT', '$,']
['die', 'der', 'PRON', 'PRELS']
['die', 'der', 'DET', 'ART']
['Dietriche', 'Dietriche', 'NOUN', 'NN']
['erfunden', 'erfinden', 'VERB', 'VVPP']
['haben', 'haben', 'AUX', 'VAFIN']
[',', ',', 'PUNCT', '$,']
['verurteilen', 'verurteilen', 'VERB', 'VVFIN']
[',', ',', 'PUNCT', '$,']
['irren', 'irren', 'VERB', 'VVFIN']
['sich', 'sich', 'PRON', 'PRF']
['.', '.', 'PUNCT', '$.']


In [2]:
def clean_string(string):
    # Replace all numbers with "0"
    p1 =re.compile('[\\.,]*[0-9]+')  
    temp = p1.sub('0',string)
    p2 =re.compile('0+')
    temp = p2.sub('0',temp)
  
    # Remove everything that is not a number or letter 
    p2 = re.compile("[^[A-Za-zÄäÖöÜüß0-9]\\s]")
    temp = p2.sub(" ",temp)

    # Shrink down to just one white space
    p3 =re.compile("[\\s]+")
    temp = p3.sub(" ", temp)
  
    return(temp)

In [7]:
reviews =csv.reader(open('reviews2.csv'))
reviews_new =csv.writer(open('reviews2_new.csv','w'))
i=0
for row in reviews:
    text =clean_string(row[0])
    doc =nlp(text)
    if i==0:
        reviews_new.writerow(['ID', 'Text', 'Lemma', 'Wortart', 'Tag'])
    else:
        for token in doc:
            reviews_new.writerow([i ,token.text, token.lemma_, token.pos_, token.tag_])
    i += 1

In [8]:
reviews2 = pd.read_csv('reviews2_new.csv').assign(Anz=1)

summary = pd.DataFrame(reviews2.groupby(['ID', 'Lemma', 'Wortart'])['Anz'].sum())
summary.to_csv('reviews_lemma1.csv')

summary = pd.DataFrame(reviews2.groupby(['ID', 'Wortart'])['Anz'].sum())
summary.to_csv('reviews_pos.csv')

summary = pd.DataFrame(reviews2.groupby(['ID', 'Tag'])['Anz'].sum())
summary.to_csv('reviews_tag.csv')


In [61]:
reviews3 = pd.read_csv('reviews2_new.csv').assign(Anz=1)
reviews3['Lemma'] = reviews3['Lemma'].mask(reviews3['Wortart'] == 'PUNCT', reviews3['Wortart'])
summary = pd.DataFrame(reviews3.groupby(['ID', 'Lemma'])['Anz'].sum())
summary.to_csv('reviews_lemma2.csv')